In [19]:
cd Downloads

/Users/georgesdelrieu/Downloads


In [20]:
ingredients = pd.read_csv('Ingredients_raw.csv')
ingredients.columns = ['Class', 'Group', 'Group_Letter_Code', 'No', 'Commodity']

In [21]:
categories = pd.read_csv('Categories_raw.csv')

In [22]:
categories.columns = ['Group_Letter_Code', 'Group', 'Class', 'Type', 'Group No', 'Page No']

In [23]:
categories.head()

,Group_Letter_Code,Group,Class,Type,Group No,Page No
0,AF,"Straw, fodder and forage of cereal grains and ...",C,11,51,119.0
1,AL,Legume animal feeds,C,11,50,116.0
2,AM,Miscellaneous Fodder and Forage crops,C,11,52,122.0
3,AR,"Frogs, lizards, snakes and turtles",B,9,48,111.0
4,AS,"Straw, fodder and forage of cereal grains and ...",C,11,51,119.0


In [24]:
ingredients['Group_Letter_Code']

0       FI
1       FI
2       FT
3       GC
4       FI
        ..
1746    WS
1747    FB
1748    MM
1749    VC
1750    VC
Name: Group_Letter_Code, Length: 1751, dtype: object

In [25]:
ingredients.head()

,Class,Group,Group_Letter_Code,No,Commodity
0,A,6.0,FI,-,AbricÃ³-da-praia
1,A,6.0,FI,-,Abyssinian gooseberry
2,A,5.0,FT,-,"Acerola, see Barbados cherry"
3,A,20.0,GC,-,"Acha, see Hungry Rice"
4,A,6.0,FI,-,"Achiote, see Annatto"


In [26]:
our_ingredients = ingredients.merge(categories, how = 'left', on = 'Group_Letter_Code')

In [27]:
our_ingredients.head()

,Class_x,Group_x,Group_Letter_Code,No,Commodity,Group_y,Class_y,Type,Group No,Page No
0,A,6.0,FI,-,AbricÃ³-da-praia,Assorted tropical and sub-tropical fruits â€“ ...,A,1.0,6.0,17.0
1,A,6.0,FI,-,Abyssinian gooseberry,Assorted tropical and sub-tropical fruits â€“ ...,A,1.0,6.0,17.0
2,A,5.0,FT,-,"Acerola, see Barbados cherry",Assorted tropical and sub-tropical fruits â€“ ...,A,1.0,5.0,11.0
3,A,20.0,GC,-,"Acha, see Hungry Rice",Cereal grains,A,3.0,20.0,57.0
4,A,6.0,FI,-,"Achiote, see Annatto",Assorted tropical and sub-tropical fruits â€“ ...,A,1.0,6.0,17.0


In [29]:
GHG_emissions = pd.read_csv('GHG_emissions.csv')

In [30]:
first_categories = GHG_emissions['Food product'].unique()

In [31]:
second_categories = our_ingredients.Group_y.unique()

In [32]:
first_categories

array(['Wheat & Rye (Bread)', 'Maize (Meal)', 'Barley (Beer)', 'Oatmeal',
       'Rice', 'Potatoes', 'Cassava', 'Cane Sugar', 'Beet Sugar',
       'Other Pulses', 'Peas', 'Nuts', 'Groundnuts', 'Soymilk', 'Tofu',
       'Soybean Oil', 'Palm Oil', 'Sunflower Oil', 'Rapeseed Oil',
       'Olive Oil', 'Tomatoes', 'Onions & Leeks', 'Root Vegetables',
       'Brassicas', 'Other Vegetables', 'Citrus Fruit', 'Bananas',
       'Apples', 'Berries & Grapes', 'Wine', 'Other Fruit', 'Coffee',
       'Dark Chocolate', 'Beef (beef herd)', 'Beef (dairy herd)',
       'Lamb & Mutton', 'Pig Meat', 'Poultry Meat', 'Milk', 'Cheese',
       'Eggs', 'Fish (farmed)', 'Shrimps (farmed)'], dtype=object)

In [33]:
second_categories

array(['Assorted tropical and sub-tropical fruits â€“ inedible peel',
       'Assorted tropical and sub-tropical fruits â€“ edible peel',
       'Cereal grains', 'Root and tuber vegetables', 'Pulses', 'Spices',
       'Marine fish', 'Legume animal feeds',
       'Fruiting vegetables, other than Cucurbits', 'Tree nuts', nan,
       'Leafy vegetables (including Brassica leafy vegetables)',
       'Oilseed', 'Berries and other small fruits', 'Freshwater fish',
       'Herbs', 'Dried herbs', 'Legume vegetables', 'Pome fruits',
       'Fruit juices', 'Dried fruits', 'Stone fruits',
       'Dried meat and fish products',
       'Molluscs (including Cephalopods) and other invertebrate animals',
       'Straw, fodder and forage of cereal grains and grasses (including buckwheat fodder)',
       'Citrus fruits', 'Fruiting vegetables, Cucurbits',
       'Fish roe (including milt = soft roe) and edible offal of fish: offal',
       'Milled cereal products (early milling stages)',
       'Brassica 

In [34]:
dic = {'Assorted tropical and sub-tropical fruits â€“ inedible peel':'Other Fruit', 
       'Assorted tropical and sub-tropical fruits â€“ edible peel':'Other Fruit',
       'Cereal grains':'Rice',
       'Root and tuber vegetables':'Root Vegetables',
       'Pulses' : 'Other Pulses',
       'Marine fish' : 'Fish (farmed)',
       'Fruiting vegetables, other than Cucurbits' : 'Other Vegetables',
       'Tree nuts' : 'Nuts',
       'Leafy vegetables (including Brassica leafy vegetables)' : 'Other Vegetables',
       'Oilseed': 'Rapeseed Oil',
       'Berries and other small fruits' : 'Berries & Grapes',
       'Freshwater fish' : 'Fish (farmed)',
       'Herbs' : 'Other Vegetables',
       'Dried herbs' : 'Other Vegetables',
       'Legume vegetables' : 'Peas',
       'Pome fruits' : 'Other Fruit',
       'Fruit juices' : 'Other Fruit',
       'Dried fruits' : 'Other Fruit',
       'Citrus fruits' : 'Citrus Fruit',
       'Fruiting vegetables, Cucurbits' : 'Other Vegetables',
       'Fish roe (including milt = soft roe) and edible offal of fish: offal' : 'Fish (farmed)',
       'Milled cereal products (early milling stages)' : 'Wheat & Rye (Bread)',
       'Brassica (cole or cabbage) vegetables, Head' : 'Brassicas',
       'Diadromous fish' : 'Fish (farmed)',
       'Milks' : 'Milk',
       'Milk fats' : 'Cheese',
       'Animal fats, processed' : 'Pig Meat',
       'Bulb vegetables' : 'Onions & Leeks',
       'Stalk and stem vegetables' : 'Other Vegetables',
       'Eggs' : 'Eggs',
       'Poultry fats' : 'Poultry Meat',
       'Poultry meat (including Pigeon meat)' : 'Poultry Meat',
       'Poultry, Edible offal of' : 'Poultry Meat',
       'Vegetable oils, crude' : 'Olive Oil',
       'Vegetable oils, edible (or refined)' : 'Olive Oil',
       'Crustaceans, processed' : 'Shrimps (farmed)',
       'Crustaceans' : 'Shrimps (farmed)',
       'Seed for beverages and sweets' : 'Barley (Beer)',
       'Grasses for sugar or syrup production': 'Cassava'
      }


In [35]:
our_ingredients['Food product'] = [dic.get(n, n) for n in our_ingredients['Group_y']]

In [36]:
to_drop = ['Spices', 'Legume animal feeds', 'Dried meat and fish products', 'Molluscs (including Cephalopods) and other invertebrate animals',
          'Straw, fodder and forage of cereal grains and grasses (including buckwheat fodder)',
          'Manufactured multi-ingredient cereal products',
          'Edible offal (mammalian)',
           'Frogs, lizards, snakes and turtles',
           'Cereal grain milling fractions', 
           'Miscellaneous Fodder and Forage crops',
           'Marine mammals',
           'Teas',
           'Dried vegetables']

In [37]:
GHG_emissions.head()

,Food product,Land use change,Animal Feed,Farm,Processing,Transport,Packging,Retail,Unnamed: 8
0,Wheat & Rye (Bread),0.1,0.0,0.8,0.2,0.1,0.1,0.1,NaN
1,Maize (Meal),0.3,0.0,0.5,0.1,0.1,0.1,0.0,NaN
2,Barley (Beer),0.0,0.0,0.2,0.1,0.0,0.5,0.3,NaN
3,Oatmeal,0.0,0.0,1.4,0.0,0.1,0.1,0.0,NaN
4,Rice,0.0,0.0,3.6,0.1,0.1,0.1,0.1,NaN


In [38]:
final_impact = our_ingredients.merge(GHG_emissions, on = 'Food product')

In [41]:
clean_ingredients = final_impact.drop(['Class_x', 'Group_x', 'Group_y', 'Class_y', 'Type', 'Group No', 'Page No', 'Unnamed: 8'], axis = 1)

In [42]:
clean_ingredients.head()

,Group_Letter_Code,No,Commodity,Food product,Land use change,Animal Feed,Farm,Processing,Transport,Packging,Retail
0,FI,-,AbricÃ³-da-praia,Other Fruit,0.1,0.0,0.4,0.0,0.2,0.0,0.0
1,FI,-,Abyssinian gooseberry,Other Fruit,0.1,0.0,0.4,0.0,0.2,0.0,0.0
2,FT,-,"Acerola, see Barbados cherry",Other Fruit,0.1,0.0,0.4,0.0,0.2,0.0,0.0
3,FI,-,"Achiote, see Annatto",Other Fruit,0.1,0.0,0.4,0.0,0.2,0.0,0.0
4,FT,-,African plum,Other Fruit,0.1,0.0,0.4,0.0,0.2,0.0,0.0


In [43]:
clean_ingredients['SumGHG'] = clean_ingredients['Land use change'] + clean_ingredients['Animal Feed'] + clean_ingredients['Farm'] + clean_ingredients['Processing'] + clean_ingredients['Transport'] + clean_ingredients['Packging'] + clean_ingredients['Retail']

In [44]:
clean_ingredients.head()

,Group_Letter_Code,No,Commodity,Food product,Land use change,Animal Feed,Farm,Processing,Transport,Packging,Retail,SumGHG
0,FI,-,AbricÃ³-da-praia,Other Fruit,0.1,0.0,0.4,0.0,0.2,0.0,0.0,0.7
1,FI,-,Abyssinian gooseberry,Other Fruit,0.1,0.0,0.4,0.0,0.2,0.0,0.0,0.7
2,FT,-,"Acerola, see Barbados cherry",Other Fruit,0.1,0.0,0.4,0.0,0.2,0.0,0.0,0.7
3,FI,-,"Achiote, see Annatto",Other Fruit,0.1,0.0,0.4,0.0,0.2,0.0,0.0,0.7
4,FT,-,African plum,Other Fruit,0.1,0.0,0.4,0.0,0.2,0.0,0.0,0.7


In [45]:
clean_ingredients.to_csv('clean_ref_ingredients.csv')